In [ ]:
import geopandas as gpd
import pandas as pd



# --- Lista dei file e delle categorie corrispondenti ---
files = {
    "../Data/Clean/Analisi/sport_tempo_libero_unito_clean.geojson": "Sport e Tempo Libero",
    "../Data/Clean/Analisi/istruzione_filtrato_unito_clean.geojson": "Istruzione",
    "../Data/Clean/Analisi/sanita_filtrato_unito_clean.geojson": "Sanità",
    "../Data/Clean/Analisi/cultura_spettacolo_filtrato_unito_clean.geojson": "Cultura e Spettacolo"
}

# --- Carico e aggiungo la colonna categoria ---
gdfs = []
for path, categoria in files.items():
    gdf = gpd.read_file(path)
    gdf["Categoria"] = categoria   # nuova colonna con il nome desiderato
    gdfs.append(gdf)

# --- Concateno tutti in un unico GeoDataFrame ---
gdf_unico = pd.concat(gdfs, ignore_index=True)

gdf_unico.to_file("../Data/Clean/Analisi/ciclabili_con_categoria_unite.geojson", driver="GeoJSON")

In [6]:
# --- Creo colonne booleane per ogni categoria ---
categorie = list(files.values())  # ["Sport e Tempo Libero", "Istruzione", "Sanità", "Cultura e Spettacolo"]

for cat in categorie:
    gdf_unico[cat] = gdf_unico["Categoria"] == cat

gdf_unico.to_file("../Data/Clean/Analisi/ciclabili_con_categoria_unite.geojson", driver="GeoJSON")
